In [2]:
import pandas as pd
import os
import numpy as np
import sklearn
import warnings
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.impute import SimpleImputer
from feature_engine.selection import SelectBySingleFeaturePerformance
from feature_engine.encoding import RareLabelEncoder, MeanEncoder, CountFrequencyEncoder, OrdinalEncoder
from feature_engine.datetime import DatetimeFeatures
from feature_engine.outliers import Winsorizer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, PowerTransformer, FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Akanksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Akanksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Akanksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernel

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Akanksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\Akanksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Akanksha\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernel

AttributeError: _ARRAY_API not found

In [3]:
pd.set_option("display.max_columns", None)
sklearn.set_config(transform_output='pandas')
warnings.filterwarnings('ignore')

In [4]:
data_dir = "C:/Users/Akanksha/Flight_Price_Prediction/data"
files = ['train', 'val', 'test']

def read_data(filename):
    filepath = os.path.join(data_dir, filename + '.csv')
    return pd.read_csv(filepath)

data = {}
for name in files:
    data[name] = read_data(name)
    
train, val, test = data['train'], data['val'], data['test']

In [5]:
train.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,SpiceJet,2019-05-24,Banglore,Delhi,21:10:00,00:05:00,175.0,0,No Info,3971
1,IndiGo,2019-04-12,Banglore,Delhi,18:55:00,21:50:00,175.0,0,No Info,5780
2,IndiGo,2019-05-03,Chennai,Kolkata,11:35:00,14:00:00,145.0,0,No Info,4332
3,Multiple Carriers,2019-06-27,Delhi,Cochin,07:00:00,19:00:00,720.0,1,No Info,12192
4,Jet Airways,2019-05-21,Kolkata,Banglore,06:30:00,16:20:00,590.0,1,No Info,14781


In [6]:
val.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,2019-04-12,Banglore,Delhi,07:10:00,10:05:00,175.0,0,No Info,4823
1,Air India,2019-05-21,Kolkata,Banglore,09:25:00,13:45:00,1700.0,2,No Info,11542
2,Air India,2019-06-24,Delhi,Cochin,09:45:00,09:25:00,1420.0,2,No Info,10975
3,Jet Airways,2019-06-12,Delhi,Cochin,09:35:00,04:25:00,1130.0,2,No Info,13292
4,IndiGo,2019-03-09,Banglore,New Delhi,16:55:00,19:45:00,170.0,0,No Info,10067


In [7]:
test.head()

,Airline,Date_of_Journey,Source,Destination,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,Jet Airways,2019-06-09,Delhi,Cochin,05:30:00,04:25:00,1375.0,2,No Info,13014
1,SpiceJet,2019-06-24,Kolkata,Banglore,22:20:00,00:40:00,140.0,0,No check-in baggage included,3841
2,Air Asia,2019-04-18,Banglore,Delhi,04:55:00,07:45:00,170.0,0,No Info,4282
3,Air India,2019-04-09,Delhi,Cochin,17:15:00,07:40:00,865.0,2,No Info,8052
4,Jet Airways,2019-06-12,Kolkata,Banglore,14:05:00,18:15:00,1690.0,1,In-flight meal not included,8529


In [8]:
#airline
air_transformer = Pipeline(steps = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('grouper', RareLabelEncoder(tol=0.10, replace_with = 'Others', n_categories=2)),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'))
])


#doj
features_to_extract = ['month', 'week', 'day_of_week', 'day_of_year']

doj_transformer = Pipeline(steps=[
    ("dt", DatetimeFeatures(features_to_extract=features_to_extract, yearfirst=True, format='mixed')),
    ('scaling', MinMaxScaler())
])

#source & destination
location_transformer = Pipeline(steps=[
    ('grouper', RareLabelEncoder(tol=0.10, replace_with = 'Others', n_categories=2)),
    ('mean_encoder', MeanEncoder()),
    ('power-transformer', PowerTransformer())
])

def is_north(X):
    columns = X.columns.to_list()
    north_cities = ['Delhi', 'Mumbai', 'Kolkata', 'New Delhi']
    
    return (
        X
        .assign(**{
            f'{col}_is_north': X.loc[:,col].isin(north_cities).astype(int)
            for col in columns
        }
        )
        .drop(columns=['Source', 'Destination'], axis=1)
    )
    
loc = FeatureUnion(transformer_list=[
    ('part1', location_transformer),
    ('part2', FunctionTransformer(func=is_north)),
]
)

#dep_time & arrival_time
time_transformer_1 = Pipeline(steps=[
    ('dt', DatetimeFeatures(features_to_extract=['hour', 'minute'])),
    ('scaler', MinMaxScaler())
])

def part_of_day(X, morning=4, afternoon=12, evening=16, night=20):
    columns = X.columns.to_list()
    X_copy = X.assign(**{
        col: pd.to_datetime(X.loc[:,col]).dt.hour
        for col in columns}
    )
    return (
        X_copy
        .assign(**{
            f"{col}_part_of_day": np.select(
                [X_copy.loc[:, col].between(morning, afternoon, inclusive='left'),
                 X_copy.loc[:, col].between(afternoon, evening, inclusive='left'),
                 X_copy.loc[:, col].between(evening, night, inclusive='left')],
                ["morning", 'afternoon', 'evening'],
                default='night'
            )
            for col in columns
        })
        .drop(columns=['Dep_Time', 'Arrival_Time'], axis=1)
    )
       
time_transformer_2 = Pipeline(steps=[
    ('pod', FunctionTransformer(func=part_of_day)),
    ('encoder', CountFrequencyEncoder()),
    ('scaler', MinMaxScaler())
])

time = FeatureUnion(transformer_list=[
    ('pipe1', time_transformer_1),
    ('pipe2', time_transformer_2)
])

#duration
def duration_groups(X):
    columns = X.columns.to_list()
    return (
        X
        .assign(**{
            f"{col}_duration_group": np.select(
                [X.loc[:, col].between(X.Duration.min(), 170.0, inclusive='left'),
                 X.loc[:, col].between(170.0, 510.0, inclusive='left'),
                 X.loc[:, col].between(510.0, 910.0, inclusive='left')],
                ['short', 'medium', 'long'],
                default='very_long'
                )
            for col in columns
        })
        .drop(columns=['Duration'], axis=1)
    )
    
dura_1 = Pipeline(steps=[
    ('duration_groups', FunctionTransformer(func=duration_groups)),
    ('encoder', OrdinalEncoder()),
])

class RBFPercentileSimilarity(BaseEstimator, TransformerMixin):
    def __init__(self, variables=None, percentiles=[0.25, 0.5, 0.75], gamma=0.1):
        self.variables=variables
        self.percentiles = percentiles
        self.gamma=gamma
    
    def fit(self, X, y=None):
        if not self.variables:
            self.variables = X.select_dtypes(include='number').columns.to_list()
             
        self.reference_values = {
            col :(
                X
                .loc[:, col]
                .quantile(self.percentiles)
                .values #converting to arrays
                .reshape(-1, 1)
            )
            for col in self.variables
        }
        
        return self

    def transform(self, X):
        objects = []
        for col in self.variables:
            columns = [f"{col}_rbf_{percentiles}" for percentiles in self.percentiles]
            obj = pd.DataFrame(
                data = rbf_kernel(
                    X.loc[:, [col]], 
                    Y=self.reference_values[col], 
                    gamma=self.gamma),
                columns=columns
            )
        objects.append(obj)
        return pd.concat(objects, axis=1)
    
duration_pipe2 = Pipeline(steps=[
    ('rbf', RBFPercentileSimilarity()),
    ('scaler', PowerTransformer()),
])

duration_union = FeatureUnion(transformer_list=[
    ('part1', dura_1),
    ('part2', duration_pipe2),
    ('part3', StandardScaler())
])

duration_transformer = Pipeline(steps=[
    ('outliers', Winsorizer(capping_method='iqr', fold=1.5)),
    ('imputer', SimpleImputer(strategy='median')),
    ('duration_union', duration_union),
])

#total_stops
def is_direct(X):
    return (
        X
        .assign(
            is_direct_flight = X.Total_Stops.eq(0)
        )
    )
    
total_stops_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy='most_frequent')),
    ("is_direct", FunctionTransformer(func=is_direct)),
])

#additional_info
def has_info(X):
    return (
        X
        .assign(
            has_info = X.Additional_Info.ne("No Info").astype(int)
        )
        .drop(columns=['Additional_Info'], axis=1)
    )
    
add_info = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="constant", fill_value="unknown")),
    ('grouper', RareLabelEncoder(tol=0.10, replace_with = 'Others', n_categories=2)),
    ('encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
    
])

info_transformer = FeatureUnion(transformer_list=[
    ('part1', add_info),
    ('part2', FunctionTransformer(func=has_info)),
])

#Column Transformer
column_transformer = ColumnTransformer(transformers=[
    ('air', air_transformer, ['Airline']),
    ('doj', doj_transformer, ['Date_of_Journey']),
    ('location', loc, ['Source', 'Destination']),
    ('time', time, ['Dep_Time', 'Arrival_Time']),
    ('duration', duration_transformer, ['Duration']),
    ('total_stops', total_stops_transformer, ['Total_Stops']),
    ('additional_info', info_transformer, ['Additional_Info'])
    ],
    remainder='passthrough'
)

estimator = RandomForestRegressor(
    n_estimators=10,
    max_depth=3,
    random_state=37
    )

selector = SelectBySingleFeaturePerformance(
    estimator=estimator,
    scoring='r2',
    threshold=0.1
)

preprocessor = Pipeline(steps=[
    ('ct', column_transformer),
    ('selector', selector)
])

In [9]:
preprocessor.fit(
    train.drop('Price', axis=1),
    train['Price'].copy()
)

Pipeline(steps=[('ct',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('air',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('grouper',
                                                                   RareLabelEncoder(n_categories=2,
                                                                                    replace_with='Others',
                                                                                    tol=0.1)),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Airline']),
                                                 ('doj',
                                                  Pipeline(steps=[('dt',
                                                                   Datetim...
                                                                                                                    replace_with='Others',
                                                                                                                    tol=0.1)),
                                                                                                  ('encoder',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse_output=False))])),
                                                                                 ('part2',
                                                                                  FunctionTransformer(func=<function has_info at 0x0000022860FAA8E0>))]),
                                                  ['Additional_Info'])])),
                ('selector',
                 SelectBySingleFeaturePerformance(estimator=RandomForestRegressor(max_depth=3,
                                                                                  n_estimators=10,
                                                                                  random_state=37),
                                                  scoring='r2',
                                                  threshold=0.1))])

In [10]:
preprocessor.transform(train.drop('Price', axis=1))

,air__Airline_IndiGo,air__Airline_Jet Airways,air__Airline_Others,doj__Date_of_Journey_week,doj__Date_of_Journey_day_of_year,location__Source,location__Destination,duration__Duration_duration_group,duration__Duration_rbf_0.25,duration__Duration,total_stops__Total_Stops,total_stops__is_direct_flight
0,0.0,0.0,1.0,0.705882,0.711864,-0.920590,-1.837680,1,2.449056,-0.917525,0,True
1,1.0,0.0,0.0,0.352941,0.355932,-0.920590,-1.837680,1,2.449056,-0.917525,0,True
2,1.0,0.0,0.0,0.529412,0.533898,-1.857672,-0.838307,0,-0.355467,-0.978436,0,True
3,0.0,0.0,0.0,1.000000,1.000000,1.045647,1.044031,2,-0.355467,0.189011,1,False
4,0.0,1.0,0.0,0.705882,0.686441,-0.181656,-0.197682,2,-0.355467,-0.074934,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...
6689,1.0,0.0,0.0,0.647059,0.661017,-0.920590,-1.837680,1,3.159405,-0.927677,0,True
6690,1.0,0.0,0.0,0.176471,0.169492,-0.920590,-0.838307,1,-0.355467,-0.663733,1,False
6691,0.0,0.0,1.0,0.588235,0.559322,1.045647,1.044031,1,-0.355467,-0.521609,1,False
6692,0.0,0.0,1.0,0.764706,0.737288,-1.857672,-0.838307,0,-0.355467,-0.988587,0,True


In [11]:
preprocessor.fit(
    train.drop('Price', axis=1),
    train['Price']
)

print(preprocessor.transform(train.drop('Price', axis=1)).shape)
print(preprocessor.transform(val.drop('Price', axis=1)).shape)
print(preprocessor.transform(test.drop('Price', axis=1)).shape)

train_pre = preprocessor.transform(train.drop('Price', axis=1))
val_pre = preprocessor.transform(val.drop('Price', axis=1))
test_pre = preprocessor.transform(test.drop('Price', axis=1))

(6694, 12)
(1674, 12)
(2093, 12)


In [12]:
val_pre.head()

,air__Airline_IndiGo,air__Airline_Jet Airways,air__Airline_Others,doj__Date_of_Journey_week,doj__Date_of_Journey_day_of_year,location__Source,location__Destination,duration__Duration_duration_group,duration__Duration_rbf_0.25,duration__Duration,total_stops__Total_Stops,total_stops__is_direct_flight
0,1.0,0.0,0.0,0.352941,0.355932,-0.920590,-1.837680,1,2.449056,-0.917525,0,True
1,0.0,0.0,0.0,0.705882,0.686441,-0.181656,-0.197682,3,-0.355467,2.178745,2,False
2,0.0,0.0,0.0,1.000000,0.974576,1.045647,1.044031,3,-0.355467,1.610250,2,False
3,0.0,1.0,0.0,0.882353,0.872881,1.045647,1.044031,3,-0.355467,1.021451,2,False
4,1.0,0.0,0.0,0.058824,0.067797,-0.920590,-0.838307,1,3.159405,-0.927677,0,True


In [13]:
test_pre.dtypes

air__Airline_IndiGo                  float64
air__Airline_Jet Airways             float64
air__Airline_Others                  float64
doj__Date_of_Journey_week            float64
doj__Date_of_Journey_day_of_year     float64
location__Source                     float64
location__Destination                float64
duration__Duration_duration_group      int64
duration__Duration_rbf_0.25          float64
duration__Duration                   float64
total_stops__Total_Stops               int64
total_stops__is_direct_flight           bool
dtype: object

In [22]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [15]:
train_labels = train['Price'].copy()
val_labels = val['Price'].copy()
test_labels = test['Price'].copy()

In [23]:
# Initial Model Training

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
xgb_reg.fit(train_pre, train_labels)

# Evaluate on the training set
train_predictions = xgb_reg.predict(train_pre)
train_rmse = np.sqrt(mean_squared_error(train_labels, train_predictions))
train_mae = mean_absolute_error(train_labels, train_predictions)
train_r2_score = r2_score(train_labels, train_predictions)
print(f"Training RMSE: {train_rmse}")
print(f"Training MAE: {train_mae}")
print(f"Training R2_Score: {train_r2_score}")

Training RMSE: 1511.0748046825902
Training MAE: 997.790316228123
Training R2_Score: 0.8915690779685974


In [24]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5, 6],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

grid_search = GridSearchCV(
    estimator=xgb_reg,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=3,
    verbose=1,
    n_jobs=-1
)

grid_search.fit(train_pre, train_labels)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=42, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.6, 0.8, 1.0],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 4, 5, 6],
                         'n_estimators': [50, 100, 200],
                         'subsample': [0.6, 0.8, 1.0]},
             scoring='neg_mean_squared_error', verbose=1)

In [25]:
best_params = grid_search.best_params_
print(f"Best parameters found: {best_params}")

Best parameters found: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200, 'subsample': 1.0}


In [26]:
final_model = xgb.XGBRegressor(**best_params)
final_model.fit(pd.concat([train_pre, val_pre]), pd.concat([train_labels, val_labels]))

# Evaluate on the validation set
val_predictions = final_model.predict(val_pre)
val_rmse = np.sqrt(mean_squared_error(val_labels, val_predictions))
val_mae = mean_absolute_error(val_labels, val_predictions)
val_r2_score = r2_score(val_labels, val_predictions)
print(f"Validation RMSE: {val_rmse}")
print(f"Validation MAE: {val_mae}")
print(f"Validation R2_Score: {val_r2_score}")

Validation RMSE: 1681.7194493078184
Validation MAE: 1178.1552130586358
Validation R2_Score: 0.8651871681213379


In [27]:
test_predictions = final_model.predict(test_pre)
test_rmse = np.sqrt(mean_squared_error(test_labels, test_predictions))
test_mae = mean_absolute_error(test_labels, test_predictions)
test_r2_score = r2_score(test_labels, test_predictions)
print(f"Test RMSE: {test_rmse}")
print(f"Test MAE: {test_mae}")
print(f"Test R2_Score: {test_r2_score}")

Test RMSE: 2085.2009556191974
Test MAE: 1326.9836652074996
Test R2_Score: 0.8086659908294678


In [21]:
print(f"Train R2 : {final_model.score(train_pre, train['Price'])}")
print(f"Validation R2 : {final_model.score(val_pre, val['Price'])}")
print(f"Test R2 : {final_model.score(test_pre, test['Price'])}")

Train R2 : 0.8509716987609863
Validation R2 : 0.8651871681213379
Test R2 : 0.8086659908294678
